In [1]:
# NumPy versions below 1.17 may be incompatible with some other 
# packages, so you may need to replace your current version with 
# an earlier one in order to run this notebook as-is. 
# !pip uninstall numpy --yes
# !pip install "numpy<1.17"

In [2]:
import numpy as np
import pandas as pd
import json
from selenium.webdriver import Chrome, Firefox
from functions import get_posts, get_hashtags, get_image, get_full_info
import boto3
import os

## Hashtags to scrape: 
1. travel
2. food
3. animals
4. selfie
5. cars
6. fitness
7. babies
8. wedding
9. nature
10. architecture

## Scraping:

### Hashtags (from comments), Image Link, Image Name, Post Link, and Input Hashtag.  Then exporting the data to .json files, stored in a seperate folder.

### *get_full_info* function also retrieves and downloads the actual image, which are stored locally to optimize possible space constraints.

### **If you want to scrape your own hashtags,** simply add them to the list and run the remaining cells. 

In [3]:
# EXAMPLE:
# hashtags = ["travel", "food", "animals", "selfie", "cars", "fitness", "babies", "wedding", "nature", "architecture"]

# Your own hashtags here:
hashtags = []

In [4]:
folder_names = ["data", "metadata"]
for folder_name in folder_names:
    try:
        os.mkdir(folder_name)
    except OSError:
        print(f"Folder '{folder_name}' already exists.")

Folder 'data' already exists.
Folder 'metadata' already exists.


In [5]:
for hashtag in hashtags:
    hashtag_data = get_full_info(hashtag, 10)

    with open(f"metadata/{hashtag}.json", "w") as f:
        json.dump(hashtag_data, f)

### You can use `pd.read_json` to import hashtag data again. 

In [6]:
# travel_df = pd.read_json("metadata/travel.json")
# travel_df.head()

### Optionally you can also use this scaffolding for uploading scraped images to an S3 bucket, although you will of course need to set up your own S3 bucket.

In [7]:
# s3 = boto3.resource("s3")

# hashtags_to_upload = ["foo", "bar"]
# for hashtag in hashtags_to_upload:
#     for img in hashtag: 
#         source = f"data/{img["image_local_name"]}"
#         bucket = f"instagram-images-mod4"
#         destination = f"{img["search_hashtag"]}/{img["image_local_name"]}"
#         s3.meta.client.upload_file(source, bucket, destination)